# Advent of Code 2018

In [18]:
import os
import re
import string

from collections import Counter, defaultdict, deque
from datetime import datetime
from itertools import accumulate, chain, combinations, cycle, zip_longest
import operator as op

import networkx as nx
import numpy as np
import pandas as pd
import requests

from scipy.spatial.distance import cdist

%matplotlib inline

In [3]:
def input_for_day(day):
    file_name = f"day{day:0>2}_input"
    if os.path.exists(file_name):
        return os.path.abspath(file_name)
    with open("session", "r") as session_file:
        response = requests.get(f"https://adventofcode.com/2018/day/{day}/input",
                                cookies={"session": session_file.read()})
    with open(file_name, "w") as input_file:
        print(response.text.strip(), file=input_file)
    return os.path.abspath(file_name)

## Day 1
### Part One
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

In [4]:
%time
with open(input_for_day(1), "r") as input_file:
    print(f'Answer: {sum(map(int, input_file))}')

Wall time: 0 ns
Answer: 538


### Part Two
What is the first frequency your device reaches twice?

In [5]:
%%time
freq_set = {0}
with open(input_for_day(1), "r") as input_file:
    for freq in accumulate(cycle(int(line) for line in input_file)):
        if freq in freq_set or freq_set.add(freq):
            print(f"Answer: {freq}")
            break

Answer: 77271
Wall time: 20.9 ms


## Day 2
### Part One
What is the checksum for your list of box IDs?

In [6]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = [Counter(n.strip()).values() for n in input_file]
checksum = sum(2 in c for c in counters) * sum(3 in c for c in counters)
print(f"Answer: {checksum}")

Answer: 5976
Wall time: 8 ms


### Part Two
What letters are common between the two correct box IDs?

In [7]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = {x: Counter(x).values() for x in (n.strip() for n in input_file)}
for x, y in combinations((n for n, c in counters.items() if any(s in c for s in (2, 3))), 2):
    if sum(a != b for a, b in zip(x, y)) == 1:
        print(f"Answer: {''.join(a for a, b in zip(x, y) if a == b)}")
        break

Answer: xretqmmonskvzupalfiwhcfdb
Wall time: 13 ms


## Day 3
### Part One
How many square inches of fabric are within two or more claims?

In [8]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    for claim in input_file:
        _, x, y, w, h = map(int, expr.match(claim).groups())
        fabric[y:y+h, x:x+w] += 1
    print(f"Answer: {np.sum(fabric > 1)}")

Answer: 117948
Wall time: 18 ms


### Part Two
What is the ID of the only claim that doesn't overlap?

In [9]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    claims = [list(map(int, expr.match(claim).groups())) for claim in input_file]
for claim in claims:
    _, x, y, w, h = claim
    fabric[y:y+h, x:x+w] += 1
for claim in claims:
    claim_id, x, y, w, h = claim
    if np.all(fabric[y:y+h, x:x+w] == 1):
        print(f"Answer: {claim_id}")
        break

Answer: 567
Wall time: 14 ms


## Day 4
### Part One
What is the ID of the guard you chose multiplied by the minute you chose?

In [10]:
%%time
entry_expr = re.compile(r"\[(.+)\]\s(.*)")
guard_expr = re.compile("Guard\s#(\d+)")

guards = defaultdict(Counter)
with open(input_for_day(4), "r") as input_file:
    lines = (entry_expr.match(entry).groups() for entry in sorted(input_file))
for timestamp, event in ((datetime.strptime(t, "%Y-%m-%d %H:%M"), e) for t, e in lines):
    if event.startswith("Guard"):
        guard = int(guard_expr.match(event).group(1))
    elif event.startswith("falls"):
        start = timestamp
    elif event.startswith("wakes"):
        duration = int((timestamp - start).total_seconds()) // 60
        guards[guard].update(Counter(start.minute + i for i in range(duration)))

_, guard = max((sum(c.values()), g) for g, c in guards.items())

print(f"Answer: {guard * guards[guard].most_common()[0][0]}")

Answer: 4716
Wall time: 19.9 ms


### Part Two
What is the ID of the guard you chose multiplied by the minute you chose? 

In [11]:
%%time
entry_expr = re.compile(r"\[(.+)\]\s(.*)")
guard_expr = re.compile("Guard\s#(\d+)")

guards = defaultdict(Counter)
with open(input_for_day(4), "r") as input_file:
    lines = (entry_expr.match(entry).groups() for entry in sorted(input_file))
for timestamp, event in ((datetime.strptime(t, "%Y-%m-%d %H:%M"), e) for t, e in lines):
    if event.startswith("Guard"):
        guard = int(guard_expr.match(event).group(1))
    elif event.startswith("falls"):
        start = timestamp
    elif event.startswith("wakes"):
        duration = int((timestamp - start).total_seconds()) // 60
        guards[guard].update(Counter(start.minute + i for i in range(duration)))

(_, minute), guard = max((c.most_common()[0][::-1], g) for g, c in guards.items())

print(f"Answer: {guard * minute}")

Answer: 117061
Wall time: 16 ms


## Day 5
### Part One
How many units remain after fully reacting the polymer you scanned?

In [12]:
%%time
def collapse(s):
    p = ["_"]
    for u in s:
        v = p[-1]
        p.pop() if v != u and v.lower() == u.lower() else p.append(u)
            
    return len(p) - 1

with open(input_for_day(5), "r") as input_file:
    print(f"Answer: {collapse(input_file.read().strip())}")

Answer: 11252
Wall time: 17 ms


### Part Two
What is the length of the shortest polymer you can produce by removing all units of exactly one type and fully reacting the result?

In [13]:
%%time
def collapse(s):
    p = ["_"]
    for u in s:
        v = p[-1]
        p.pop() if v != u and v.lower() == u.lower() else p.append(u)
            
    return len(p) - 1

with open(input_for_day(5), "r") as input_file:
    data = input_file.read().strip()
min_length = min(collapse(s) for s in (data.replace(l, "").replace(l.swapcase(), "") for l in string.ascii_lowercase))
print(f"Answer: {min_length}")

Answer: 6118
Wall time: 246 ms


## Day 6
### Part One
What is the size of the largest area that isn't infinite?

In [14]:
%%time
with open(input_for_day(6), "r") as input_file:
    points = np.fromiter(chain.from_iterable(map(int, l.strip().split(",")) for l in input_file), np.uint).reshape(50, 2)

# Boundaries
tl = np.min(points, axis=0)
br = np.max(points, axis=0) + 1

# Index grid
xx, yy = np.mgrid[tl[0]:br[0], tl[1]:br[1]]
positions = np.vstack([xx.ravel(), yy.ravel()]).T

# Manhatten distances
dists = cdist(positions, points, metric='cityblock')
min_dists = np.min(dists, axis=1)

# Mapping to pointts
index_array = (min_dists[..., np.newaxis] == dists)

# Non ambigious distances
non_ambigious_index = (np.sum(index_array, axis=1) == 1)
index_array = index_array[non_ambigious_index]

# Borders to filter infinit regions
border_index = np.any(np.vstack([tl, br-1])[:,np.newaxis] == positions, axis=(0, 2))
border_index = border_index[non_ambigious_index]
border_index = np.any(index_array[border_index], axis=0)

# Compute arrea for all points
area = np.sum(index_array, axis=0)

# Remove points that areas touch the borders
area[border_index] = -1

print(f"Answer: {np.max(area)}")

Answer: 4754
Wall time: 47.5 ms


### Part Two
What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

In [15]:
%%time
with open(input_for_day(6), "r") as input_file:
    points = np.fromiter(chain.from_iterable(map(int, l.strip().split(",")) for l in input_file), np.uint).reshape(50, 2)

# Boundaries
tl = np.min(points, axis=0)
br = np.max(points, axis=0) + 1

# Index grid
xx, yy = np.mgrid[tl[0]:br[0], tl[1]:br[1]]
positions = np.vstack([xx.ravel(), yy.ravel()]).T

# Manhatten distances
dists = cdist(positions, points, metric='cityblock')

print(f"Answer: {np.sum(np.sum(dists, axis=1) < 10000)}")

Answer: 42344
Wall time: 25.9 ms


## Day 7
### Part One
In what order should the steps in your instructions be completed?

In [16]:
%%time
with open(input_for_day(7), "r") as input_file:
    edges = [re.findall("\s(\w)\s", edge) for edge in input_file]
vertices = set(chain.from_iterable(edges))

G = nx.DiGraph()
G.add_nodes_from(vertices)
G.add_edges_from(edges)

result = []
while G.nodes:
    n = min(G.in_degree(G.nodes), key=lambda x: (x[1],x[0]))[0]
    G.remove_node(n)
    result.append(n)

print(f"Answer: {''.join(result)}")

Answer: JNOIKSYABEQRUVWXGTZFDMHLPC
Wall time: 6.98 ms


### Part Two
With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?

In [75]:
%%time
with open(input_for_day(7), "r") as input_file:
    edges = [re.findall("\s(\w)\s", edge) for edge in input_file]
vertices = set(chain.from_iterable(edges))

G = nx.DiGraph()
G.add_nodes_from((v, {"steptime": ord(v) - 4}) for v in vertices)
G.add_edges_from(edges)

tick = 0
counts = nx.get_node_attributes(G, "steptime")
while G.nodes:
    ready_nodes = [x[0] for x in sorted(G.in_degree(G.nodes), key=lambda x: (x[1],x[0])) if x[1] == 0]
    refresh = False
    while not refresh:
        tick = tick + 1
        for node in ready_nodes[:5]:
            counts[node] = counts[node] - 1
            if counts[node] == 0:
                G.remove_node(node)
                refresh = True

print(f"Answer: {tick}")

Answer: 1099
Wall time: 1.99 ms
